In [1]:
from rdkit import Chem
import pandas as pd
from rdkit.Chem.rdmolfiles import SDMolSupplier
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import re
import random
IPythonConsole.drawOptions.addAtomIndices = True

In [ ]:
"""
This program shows how to extract a library of bipyridine sidechains from an open-source csv file containg about 
10 000 bipyridine ligand and writing those sidechains as smiles in a text file to re-used later (part 1). 
Then it shows how to use this bipyridine library to generate a large amount of cartesian coordinates of Re(bpy)(CO)3 
complexes with random side-chains (part 2), to be optimized by semi-empirical methods in a later program.
In the long run, this program is a key component of a larger program aiming at optimizing the side-chains of 
Re-bipy complexes for optimal CO2 reduction reactivity, using both QM and Machine Learning.
"""

In [203]:
"""
functions used for part 1:
"""

def MolWithoutIsotopesToSmiles(mol):
    
    """
    Returns a smile string.
    Function taken from an outside source. removes the isotope information which is unnecessary and leads our 
    algorithm to mistakenly assign same molecules as different.
    Parameter:
    mol (rdkit mol object): 
    """
    
    atom_data = [(atom, atom.GetIsotope()) for atom in mol.GetAtoms()]
    for atom, isotope in atom_data:
        if isotope:
            atom.SetIsotope(0)
    smiles = Chem.MolToSmiles(mol)
    return smiles

In [569]:
"""
We have downloaded from PubChem a large amount of bipyridine compound in the form of a csv file. Luckily, the smiles
are available in this dataset, so they can be directly converted into rdkit mol objects. This is what this code is 
doing.
"""
df = pd.read_csv('/Users/tarrago/Downloads/PubChem_compound_text_bipyridine_summary.csv')
suppl = df['isosmiles'].values
mol = []
i=0
for m1 in suppl:
    Dumdum = Chem.MolFromSmiles(m1)
    mol.append(Dumdum)
len(mol)

RDKit WARNING: [10:43:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:45] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:46] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:46] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:46] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:46] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [10:43:46] Explicit valence

11410

In [570]:
"""
The error message "WARNING: not removing hydrogen atom without neighbors" is irrelevant. we will remove those 
hydrogens in the following code. 
The list mol contains 11410 objects. However, some of these objects are 2,2'-bipyridine counted twice, some of them 
don't actually contain bipyridines, some contain bipyridines but not 2,2-bipyridines and some of them contain
two molecules or more. In the next step, we want to get rid of the intruder molecules. Here, we remove: 
- empty objects
- all objects not containing a 2,2'-bipyridine motif
- if there are more than one molecule in the object, only one molecule containing a bipyridine is kept and all other 
molecules are discarded.

This reduces our number of moleculaes do
"""

sub = Chem.MolFromSmiles('C1=CC=NC(=C1)C2=CC=CC=N2')
i=0
while i < len(mol):
    if str((type(mol[i]))) != "<class 'rdkit.Chem.rdchem.Mol'>":
        del mol[i]
    elif len(mol[i].GetSubstructMatches(sub)) == 0:
        del mol[i]
    else:
        S1=Chem.MolToSmiles(mol[i])
        if "." in S1:
            Split = S1.split('.')
            for j in Split:
                dumdum = Chem.MolFromSmiles(j)
                if len(dumdum.GetSubstructMatches(sub)) == 1:
                    mol.append(dumdum)
            del mol[i]
        else:
            i = i+1
len(mol)

RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:53] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:54] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:55] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:55] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [10:43:55] WARNING: not r

7770

In [571]:
"""
For simplicity, we also want to remove objects containing more than one bipyridine motif to avoid treatment of 
overly complex molecules.
"""
i = 0
while i < len(mol):
    if len(mol[i].GetSubstructMatches(sub)) > 1 :
        mol.remove(mol[i])
    else:
        i = i+1
len(mol)

6613

In [572]:
"""
The library contains duplicated molecules. In the following, we systematically erase those duplicates. 
"""
smilslist = []
for i in mol:
    smilslist.append(Chem.MolToSmiles(i))
for i in range(len(mol)):
    j = i+1
    while j < len(mol):
        if smilslist[i] == smilslist[j]:
            del mol[j]
            del smilslist[j]
        else:
            j = j+1
len(mol)

5237

In [573]:
"""Now, the next step is to remove the bipyridine motif in each case and save the side chains as mol objects in a list.
Then, we remove the duplicates (many duplicates will exist due to  most of these bipyridines having two identical 
side-chaines)"""

chainlib = []
i = 0
for m1 in mol:
    tmp = Chem.ReplaceCore(m1,sub,labelByIndex=True)
    try:
        rs = Chem.GetMolFrags(tmp,asMols=True)
        i = i+1
        for rs1 in rs:
            chainlib.append(rs1)
    except:
        i = i+1
smilslist = []
for i in chainlib:
    smilestext = MolWithoutIsotopesToSmiles(i)           # Homebrew function
    smilslist.append(smilestext)
for i in range(len(chainlib)):
    j = i+1
    while j < len(chainlib):
        if smilslist[i] == smilslist[j]:
            del chainlib[j]
            del smilslist[j]
        else:
            j +=1
len(chainlib)

RDKit ERROR: [10:44:56] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:57] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:58] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:58] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:58] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:58] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:58] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:58] non-ring atom 0 marked aromatic
RDKit ERROR: [10:44:58] non-ring atom 0 marked aromatic


3183

In [575]:
"""
The errors "RDKit ERROR: [time] non-ring atom 0 marked aromatic" appear whenever the sidechains of the bipyridine 
motif are in fact a ring sharing a bond with a pyridine. Such cases can be ignored, since we are only interested in
sidechains with a unique attachment point.
In the end, we get a library of sidechains which attachment points are marked by asterisk (see example below). 
For the last step, we discard the sidechains with more than one attachment point for simplicity reasons.
"""
search_and_destroy=[]
for i in chainlib:
    if len(i.GetSubstructMatches(Chem.MolFromSmiles('[*]'))) > 1:
        search_and_destroy.append(i)
for i in search_and_destroy:
    chainlib.remove(i)

len(chainlib)

3049

In [576]:
"""
The library is now set. We now need to write it in a text file. This is quite easy:
"""
root = '/Users/tarrago/Documents/personal-project/sidechain_smiles'
text_file = open(root,'w+')
for i in chainlib:
    text_file.write(Chem.MolToSmiles(i)+'\n')
text_file.close()


In [247]:
"""
In the rest of the code, we show how to use the side chains library to build a randomly-generated Re-bpy complex,
and optionally write the associated xyz files.
"""


In [577]:
"""
Functions: The following functions are related to building a random [Re(bpy)(CO)3] as an rdkit mol object, using
the sidechain library.
"""

def spawn_Rebpy(chain_library,nchains,nrep):

    """
    This function returns a list molvec containing a user-specified number of randomly generated rhenium bipyridine 
    complexes as rdkit mol objects.
    Parameters:
    chain_library (list of rkit mol objects): side-chains library. Must contain sidechains which attachment points
    aremarked by isotop carbons.
    nchains (integer): number of sidechains
    nrep (integer): number of mol objects to be created
    """

    molvec=[]
    for j in range(0,nrep):
        pickabranch=[0,1,2,5,7,8,9,10]
        Vecchain=[]
        Vecbranch=[]
        Randtrace=[]
        for i in range(0,nchains):
            Randnum=random.randint(0,len(chain_library)-1)
            Randbranch=random.choice(pickabranch)
            Vecchain.append(chain_library[Randnum])
            Vecbranch.append(Randbranch)
            pickabranch.remove(Randbranch)
        mol=mybipybuilder(Vecchain,Vecbranch)        # homebrew function
        molvec.append(mol)
    
    return molvec


def mybipybuilder(sidechains,nbrancher):
   
    """
    This function returns a Re(bpy)(CO)3 complex under the form of a rdkit mol object. The molecule is created by 
    attaching the side-chains specfied in a list to a Re(bpy)(CO)3 template without sidechains, to atoms of the bpy 
    which index is specified in a list.
    Parameters:
    sidechains(list of rdkit mol objects): list containing the side-chains as rdkit mol objects. contain an isotope 
    carbon in place of the attachment point.
    nbrancher(list of integers): list containing the index of the bipyridine atoms which must be attached to the side-
    chains.
    """
    
    template = Chem.MolFromSmiles('c1cc[n+]2c(c1)-c1cccc[n+]1[Re]2(C#[O+])(C#[O+])(C#[O+])')
    for i in range(len(sidechains)):
        side_chain = sidechains[i]
        brancher = nbrancher[i]
        template = Chem.CombineMols(template, side_chain)
        list_attachment = template.GetSubstructMatches(Chem.MolFromSmiles('[*]'))
        real_attachment = template.GetAtomWithIdx(list_attachment[0][0]).GetNeighbors()[0].GetIdx()
        mol_edit = Chem.EditableMol(template)
        mol_edit.AddBond(real_attachment, brancher)
        mol_edit.RemoveAtom(list_attachment[0][0])
        template = mol_edit.GetMol()
    template_smiles = Chem.MolToSmiles(template)
    template_smiles = template_smiles.replace('~','')
    template = Chem.MolFromSmiles(template_smiles)
    return template
     

In [578]:
"""
The following functions are related to the generation of 3D cartesian coordinates for Re(bpy) complexes for DFT 
calculations.
"""

def write_xyzfiles(chainlib,nchains,nrep,place):
    
    """
    This function randomly generates a user-specified number of Re(bpy)(CO)3 complexes and writes the corresponding 3D
    coordinates (N.B: a poor starting geometry, further optimizations are needed), a file containing the 
    smiles of the complexes, and a file containing the overall charges of each complexes, all in a unique,  
    user-defined directory. It then returns a list containing the Re(bpy)(CO)3 complexes as mol objects. 
    Parameters:
    chainlib (list of rdkit mol objects): list containing the side-chains
    nchains (integer): the number of side-chains to the complex
    nrep (integer): number of complexes to build
    place(string): the path to the directory where the xyz files must be stored.
    """
    
    try:
        sample=spawn_Rebpy(chainlib,nchains,nrep)
    except:
        pass
    indx=0
    place2=place
    for mol in sample:
        xyz_generator(mol,place,indx)            # Homebrew function
        indx=indx+1
    smiles_generator(place2,sample)              # Homebrew function
    return sample


def xyz_generator(mol,place,indx):  
    
    """
    Writes the 3D cartesian coordinate associated with a rdkit mol object in a specified location. Of note,
    the 3D structure is obtained by embedding using the ETKDG method as implemented in Rdkit. Note that the starting geometry 
    is nowhere near an equilibrium geometry and will require an additionnal geometry optimization. 
    FF optimization with Re is not implemented in Rdkit unfortunately. However, the embedding allows a starting geometry 
    with not-too-awful angles and bond lengths, and most importantly, no overlapping atoms.
    This way, with the appropriate scripts/QM/FF package, the starting geometry can be directly optimized without 
    additional user input.
    Parameters:
    mol (rdkit mol object): molecule which cartesian coordinates need to be written
    place (str): the path to the desired location
    indx (int): this is the index referencing the molecule, appearing in the xyzfile name.
    """
    
    if type(mol) is not None:
        place=place+'/Rebipy-'+str(indx)+'.xyz'
        mol2=Chem.AddHs(mol)
        N=mol2.GetNumAtoms()
        AllChem.EmbedMolecule(mol2,randomSeed=0xf00d)
        print(Chem.MolToMolBlock(mol2),file=open(place,'w+'))
        file=open(place,'r')
        lines=file.readlines()
        file.close()
        file=open(place,'w+')
        file.write(str(N)+'\n')
        file.write('This xyz file has been randomly generated!\n')
        for i in range (4,N+4):
            linesp=lines[i].split()
            linetowrite=linesp[3]+' '+linesp[0]+' '+linesp[1]+' '+linesp[2]+'\n'
            file.write(linetowrite)
    else:
        place=place+'/Rebipy-'+str(indx)+'.xyz'
        file=open(place,'w+')
        file.write('Empty mol object :(')
        print('ho-ho, we have a problem...')
    file.close


def smiles_generator(place,samp):
    
    """
    Writes in a specified location a file containing the smiles corresponding to each rdkit mol object in the input
    list. Also writes a file containing the overall charge of each complex in the same location.
    Parameters:
    place (string): the path to the directory where the xyz files must be stored.
    samp (list of rdkit mol objects): the list of complexes (as rdkit mol objects) for which smiles and charge must be
    written. 
    """
    
    placeX=place+'/smiles-index'
    file1=open(placeX,'w+')
    for sa in samp:
        file1.write(Chem.MolToSmiles(sa)+'\n')
    file1.close()
    placeY=place+'/chargefile'
    file1=open(placeY,'w+')
    for sa in samp:
        file1.write(str(Chem.GetFormalCharge(sa) - 5)+'\n')
    file1.close()




In [579]:
"""
to obtain a set of molecules in a given directory, one does simply call the write_xyzfiles function. For instance, 
to write 50 coordinate files:
"""
root='/Users/tarrago/Documents/personal-project'
nchains=2
nrep=50
random_library=write_xyzfiles(chainlib, nchains, nrep, root)

RDKit ERROR: [10:45:28] UFFTYPER: Unrecognized atom type: Re6 (43)
RDKit ERROR: [10:45:29] UFFTYPER: Unrecognized atom type: Re6 (21)
RDKit ERROR: [10:45:29] UFFTYPER: Unrecognized atom type: Re6 (25)
RDKit ERROR: [10:45:29] UFFTYPER: Unrecognized atom type: Re6 (39)
RDKit ERROR: [10:45:29] UFFTYPER: Unrecognized atom type: Re6 (79)
RDKit ERROR: [10:45:30] UFFTYPER: Unrecognized atom type: Re6 (33)
RDKit ERROR: [10:45:30] UFFTYPER: Unrecognized atom type: Re6 (2)
RDKit ERROR: [10:45:30] UFFTYPER: Unrecognized atom type: Re6 (24)
RDKit ERROR: [10:45:30] UFFTYPER: Unrecognized atom type: Re6 (27)
RDKit ERROR: [10:45:31] UFFTYPER: Unrecognized atom type: Re6 (43)
RDKit ERROR: [10:45:31] UFFTYPER: Unrecognized atom type: Re6 (2)
RDKit ERROR: [10:45:31] UFFTYPER: Unrecognized atom type: Re6 (22)
RDKit ERROR: [10:45:31] UFFTYPER: Unrecognized atom type: Re6 (58)
RDKit ERROR: [10:45:32] UFFTYPER: Unrecognized atom type: Re6 (32)
RDKit ERROR: [10:45:32] UFFTYPER: Unrecognized atom type: Re6 (3

In [545]:
"""
As mentionned earlier, the Re atom is not recognized by the algorithm, hence the complaint 
"Unrecognized atom type: Re6". In this case, the embedding procedure uses a "default" atom type. Again, in our
case this is not a problem since the structure will be optimized using more advanced methods in a later program.
"""

In [537]:
"""
Hence this program enables to rapidly generate the rdkit mol objects and cartesian coordinates for a large amount 
of randomly generated Rebpy complexes. Its purpose is to create starting structure for machine learning training set 
(structures which geometry will then be optimized and molecular properties extracted by more advanced methods). It 
also enables the rapid generation of new molecules in order to scan the chemical space (using a genetic algorithm
which will be presented in a later notebook.)
"""